### G_EVAL 테스트

- `g_eval_test_dataset_generation.ipynb`의 결과로 생성된 `financial_g_eval_test_dataset.csv`를 준비해주세요.

- 준비된 테스트 데이터셋을 리트리버와 생성 부분을 나눠서 평가하게 됩니다.

- 평가 기준은 프롬프트에 명시되어 있으며, T/F 방식으로 해당 점수를 획득 및 차감합니다.

- 각 데이터마다 평가를 하게 되며 평균을 내어 전체 점수를 계산합니다.

In [ ]:
import pandas as pd
import requests
import json
from openai import OpenAI
from typing import Dict, List
import time

def call_api(question: str) -> Dict:
    """API 호출하여 context와 answer 받기"""
    try:
        response = requests.post(
            "http://localhost:8090/api/query",
            json={"query": question},
            timeout=120
        )
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"API 호출 에러: {str(e)}")
        return None

def evaluate_retrieval(contexts: List[str], ground_truth: str, question: str, client: OpenAI) -> Dict:
    """Retrieval 평가"""
    prompt = f"""
다음 기준에 따라 검색된 컨텍스트(retrieved contexts)를 평가해주세요.

질문: {question}
Ground Truth 답변: {ground_truth}

검색된 컨텍스트:
---
{json.dumps(contexts, ensure_ascii=False, indent=2)}
---

각 평가 기준에 대해 Yes(true) 또는 No(false)로 답변해주세요:

1. Strong Similarity (5점):
컨텍스트들 중 Ground Truth와 강한 유사성을 보이는 것이 있나요?

2. Essential Information (5점):
컨텍스트들이 Ground Truth의 핵심 정보를 포함하고 있나요?

3. Question Addressing (4점):
컨텍스트들이 사용자의 질문을 충분히 다루고 있나요?

4. Relevance (3점):
모든 컨텍스트가 Ground Truth 또는 질문과 관련이 있나요?

5. Reasonable Length (3점):
컨텍스트의 길이와 수가 적절하며 불필요한 정보로 사용자를 압도하지 않나요?
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "당신은 정보 검색 시스템의 성능을 평가하는 전문가입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "retrieval_evaluation",
                "strict": True,
                "schema": {
                    "type": "object",
                    "properties": {
                        "criteria_scores": {
                            "type": "object",
                            "properties": {
                                "strong_similarity": {"type": "boolean"},
                                "essential_information": {"type": "boolean"},
                                "question_addressing": {"type": "boolean"},
                                "relevance": {"type": "boolean"},
                                "reasonable_length": {"type": "boolean"}
                            },
                            "required": ["strong_similarity", "essential_information", 
                                       "question_addressing", "relevance", "reasonable_length"],
                            "additionalProperties": False
                        }
                    },
                    "required": ["criteria_scores"],
                    "additionalProperties": False
                }
            }
        }
    )
    
    result = json.loads(response.choices[0].message.content)
    scores = result["criteria_scores"]
    
    # boolean을 점수로 변환
    score_weights = {
        "strong_similarity": 5,
        "essential_information": 5,
        "question_addressing": 4,
        "relevance": 3,
        "reasonable_length": 3
    }
    
    total_score = sum(score_weights[key] for key, value in scores.items() if value)
    
    return {
        "scores": scores,
        "total": total_score
    }

def evaluate_generation(answer: str, ground_truth: str, question: str, client: OpenAI) -> Dict:
    """Generation 평가"""
    prompt = f"""
다음 기준에 따라 생성된 답변을 평가해주세요.

질문: {question}
Ground Truth 답변: {ground_truth}
생성된 답변: {answer}

각 평가 기준에 대해 Yes(true) 또는 No(false)로 답변해주세요:

1. Relevance (5점): 답변이 질문과 명확하게 관련이 있고 사용자의 의도를 반영하나요?
2. Factual Correctness (5점): 답변이 사실적으로 정확하고 근거 없는 정보가 없나요?
3. Completeness (5점): 답변이 질문과 ground truth에서 요구하는 모든 핵심 포인트를 포함하나요?
4. Clarity (5점): 답변이 명확하고 간결하며 불필요한 반복이나 모호성이 없나요?
5. Consistency (3점): 답변이 논리적으로 구성되어 있고 맥락과 일관되며 모순이 없나요?
6. Detail Level (3점): 답변이 질문에 대해 충분한 세부 사항을 제공하되 과도하지 않나요?
7. Citations (2점): 답변이 데이터나 주장을 참조할 때 적절한 출처나 표시를 제공하나요?
8. Format (1점): 답변이 질문에 적합한 형식으로 제시되어 있나요?
9. Extra Insights (1점): 답변이 사실적 정확성을 유지하면서 유용한 추가 통찰이나 맥락을 제공하나요?
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "당신은 텍스트 생성 시스템의 성능을 평가하는 전문가입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "generation_evaluation",
                "strict": True,
                "schema": {
                    "type": "object",
                    "properties": {
                        "criteria_scores": {
                            "type": "object",
                            "properties": {
                                "relevance": {"type": "boolean"},
                                "factual_correctness": {"type": "boolean"},
                                "completeness": {"type": "boolean"},
                                "clarity": {"type": "boolean"},
                                "consistency": {"type": "boolean"},
                                "detail_level": {"type": "boolean"},
                                "citations": {"type": "boolean"},
                                "format": {"type": "boolean"},
                                "extra_insights": {"type": "boolean"}
                            },
                            "required": ["relevance", "factual_correctness", "completeness",
                                       "clarity", "consistency", "detail_level", "citations",
                                       "format", "extra_insights"],
                            "additionalProperties": False
                        }
                    },
                    "required": ["criteria_scores"],
                    "additionalProperties": False
                }
            }
        }
    )
    
    result = json.loads(response.choices[0].message.content)
    scores = result["criteria_scores"]
    
    # boolean을 점수로 변환
    score_weights = {
        "relevance": 5,
        "factual_correctness": 5,
        "completeness": 5,
        "clarity": 5,
        "consistency": 3,
        "detail_level": 3,
        "citations": 2,
        "format": 1,
        "extra_insights": 1
    }
    
    total_score = sum(score_weights[key] for key, value in scores.items() if value)
    
    return {
        "scores": scores,
        "total": total_score
    }

In [ ]:
# 데이터셋 로드
df = pd.read_csv('financial_test_dataset.csv')
client = OpenAI()

# 결과를 저장할 리스트
results = []

# 각 질문에 대해 평가 수행
for idx, row in df.iterrows():
    print(f"평가 진행 중: {idx + 1}/100")
    
    # API 호출
    api_response = call_api(row['question'])
    if not api_response:
        continue
        
    # Retrieval 평가
    retrieval_scores = evaluate_retrieval(
        api_response['context'],
        row['groundtruth'],
        row['question'],
        client
    )
    
    # Generation 평가
    generation_scores = evaluate_generation(
        api_response['answer'],
        row['groundtruth'],
        row['question'],
        client
    )
    
    # 결과 저장
    results.append({
        'id': row['id'],
        'question': row['question'],
        'groundtruth': row['groundtruth'],
        'api_answer': api_response['answer'],
        'retrieval_score': retrieval_scores['total'],
        'generation_score': generation_scores['total'],
        'total_score': retrieval_scores['total'] + generation_scores['total'],
        'retrieval_details': retrieval_scores,
        'generation_details': generation_scores
    })
    
    # API 호출 간 간격
    time.sleep(1)

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# 평균 점수 계산
avg_retrieval = results_df['retrieval_score'].mean()
avg_generation = results_df['generation_score'].mean()
avg_total = results_df['total_score'].mean()

print(f"\n평가 결과:")
print(f"Retrieval 평균 점수: {avg_retrieval:.2f}/20")
print(f"Generation 평균 점수: {avg_generation:.2f}/30")
print(f"총점 평균: {avg_total:.2f}/50")

# 결과 저장
results_df.to_csv('geval_results.csv', index=False)
print("\n상세 결과가 'geval_results.csv'에 저장되었습니다.")